In [50]:
using Pkg;
Pkg.add("StatsBase")
Pkg.add("DataFrames")
Pkg.add("CSV")

 Resolving package versions...
  Updating `/opt/julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `/opt/julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `/opt/julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `/opt/julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
 Installed WeakRefStrings ─ v0.6.2
 Installed FilePathsBase ── v0.7.0
 Installed CSV ──────────── v0.5.23
  Updating `/opt/julia/environments/v1.3/Project.toml`
  [336ed68f] + CSV v0.5.23
  Updating `/opt/julia/environments/v1.3/Manifest.toml`
  [336ed68f] + CSV v0.5.23
  [48062228] + FilePathsBase v0.7.0
  [ea10d353] + WeakRefStrings v0.6.2


In [51]:
using StatsBase
using DataFrames
using CSV

┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1273


In [29]:
mutable struct NodeJu
  key::Int64
  mutate_rate::Float64
  possible_bias_values::Array{Int64,1}
  bias::Float64
  function NodeJu(key::Int, mutate_rate::Float64, possible_bias_values::Array{Int64,1})
    node = new(key, mutate_rate, possible_bias_values, 0.0)
    node
  end
end


function mutate_bias!(node::NodeJu)
  if rand() < mutate_rate
    node.bias = StatsBase.sample(node.possible_bias_values);
  end
  node 
end


function initialize_nodes_julia(n_nodes::Int, mutate_rate::Float64, possible_bias_values::Array{Int64,1})
    nodes = Dict()
    for i = 1 : n_nodes
        node = NodeJu(i, 
                      mutate_rate, 
                      possible_bias_values)
        mutate_bias!(node)
        nodes[i] = node
    end
    nodes
end

initialize_nodes_julia (generic function with 1 method)

In [30]:
mutate_rate = 0.5
possible_bias_values = collect(range(0, length=10))

10-element Array{Int64,1}:
 0
 1
 2
 3
 4
 5
 6
 7
 8
 9

In [36]:
n_nodes = 5000000

5000000

In [37]:
a = @time initialize_nodes_julia(n_nodes, mutate_rate, possible_bias_values);

  3.928333 seconds (33.76 M allocations: 936.860 MiB, 29.66% gc time)


In [41]:
a = @elapsed initialize_nodes_julia(n_nodes, mutate_rate, possible_bias_values);
a

4.01270395

In [42]:
function run_experiments(n_nodes_values, n_rep)
    n_nodes_list = []
    elapsed_list = []

    for n_nodes in n_nodes_values
        for rep = 1:n_rep
            push!(elapsed_list, @elapsed initialize_nodes_julia(n_nodes, mutate_rate, possible_bias_values));
            push!(n_nodes_list, n_nodes)
        end
        
    end
    datadict = Dict("n_nodes" => n_nodes_list, "elapsed" => elapsed_list)
    df = DataFrame()
    for (key, value) in datadict
           df[Symbol(key)] = value
    end
    df
end

run_experiments (generic function with 1 method)

In [55]:
n_nodes_values=[1, 10, 100, 1000, 10000, 100000, 1000000, 10000000]
df = run_experiments(n_nodes_values, 10)
CSV.write("./pure_julia.csv",df)
tail(df)

┌ Warning: `tail(df::AbstractDataFrame)` is deprecated, use `last(df, 6)` instead.
│   caller = top-level scope at In[55]:4
└ @ Core In[55]:4


,elapsed,n_nodes
,Any,Any
1,10.518,10000000
2,10.0278,10000000
3,10.0673,10000000
4,10.0979,10000000
5,10.5074,10000000
6,10.2464,10000000
